<h1><strong><u>Vectorization Pipeline Test</u></strong><h1>

In [41]:
import spacy

In [42]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "talk.religion.misc",
]

data_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

print(f"Loading 20 newsgroups dataset for {len(data_train.target_names)} categories:")
print(data_train.target_names)
print(f"{len(data_train.data)} documents")
print(f"{len(data_test.data)} documents")
print(data_train.data)



Loading 20 newsgroups dataset for 2 categories:
['alt.atheism', 'talk.religion.misc']
857 documents
570 documents
['Benedikt Rosenau writes, with great authority:\n\n\n"Contradictory" is a property of language.  If I correct this to\n\n\n      THINGS DEFINED BY CONTRADICTORY LANGUAGE DO NOT EXIST\n\nI will object to definitions as reality.  If you then amend it to\n\n      THINGS DESCRIBED BY CONTRADICTORY LANGUAGE DO NOT EXIST\n\nthen we\'ve come to something which is plainly false.  Failures in\ndescription are merely failures in description.\n\n(I\'m not an objectivist, remember.)\n\n', '\n[...stuff deleted...]\n\nComputers are an excellent example...of evolution without "a" creator.\nWe did not "create" computers.  We did not create the sand that goes\ninto the silicon that goes into the integrated circuits that go into\nprocessor board.  We took these things and put them together in an\ninteresting way. Just like plants "create" oxygen using light through \nphotosynthesis.  It\'s 

In [43]:
b = data_train.data
c = data_train.target

In [44]:
nlp = spacy.load("en_core_web_sm")
def custom_spacy_tokenizer(text):
        doc = nlp(text)
        return [token.lemma_.lower().strip() for token in doc if not token.is_stop and not token.is_punct]

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ("vect", None),
        ("clf", ComplementNB()),
    ]
)
pipeline

Pipeline(steps=[('vect', None), ('clf', ComplementNB())])

In [46]:
import numpy as np

parameter_grid = [
    {"vect": [TfidfVectorizer()],
     # "vect__tokenizer": [custom_spacy_tokenizer],
     "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
     "vect__min_df": (1, 3, 5, 10),
     "vect__ngram_range": ((1, 1), (1, 2)),
     "vect__norm": ("l1", "l2"),
     "vect__use_idf": (True, False),
     "clf__alpha": np.logspace(-6, 6, 13)},
    {"vect": [CountVectorizer()],
     "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
     "vect__min_df": (1, 3, 5, 10),
     "vect__ngram_range": ((1, 1), (1, 2)),
     "clf__alpha": np.logspace(-6, 6, 13)}
    ]

In [47]:
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=parameter_grid,
    n_iter=40,
    random_state=0,
    n_jobs=2,
    verbose=1,
)

print("Performing grid search...")
print("Hyperparameters to be evaluated:")
pprint(parameter_grid)

Performing grid search...
Hyperparameters to be evaluated:
[{'clf__alpha': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
  'vect': [TfidfVectorizer()],
  'vect__max_df': (0.2, 0.4, 0.6, 0.8, 1.0),
  'vect__min_df': (1, 3, 5, 10),
  'vect__ngram_range': ((1, 1), (1, 2)),
  'vect__norm': ('l1', 'l2'),
  'vect__use_idf': (True, False)},
 {'clf__alpha': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
  'vect': [CountVectorizer()],
  'vect__max_df': (0.2, 0.4, 0.6, 0.8, 1.0),
  'vect__min_df': (1, 3, 5, 10),
  'vect__ngram_range': ((1, 1), (1, 2))}]


In [48]:
from time import time

t0 = time()
random_search.fit(data_train.data, data_train.target)
print(f"Done in {time() - t0:.3f}s")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Done in 9.481s


In [49]:
print("Best parameters combination found:")
best_parameters = random_search.best_estimator_.get_params()
for param_name in sorted(parameter_grid):
    print(f"{param_name}: {best_parameters[param_name]}")

Best parameters combination found:


TypeError: '<' not supported between instances of 'dict' and 'dict'

In [ ]:
random_search.best_params_

{'vect__use_idf': True,
 'vect__norm': 'l1',
 'vect__ngram_range': (1, 1),
 'vect__min_df': 1,
 'vect__max_df': 0.2,
 'vect': TfidfVectorizer(),
 'clf__alpha': np.float64(0.001)}

Exception ignored in: <function ResourceTracker.__del__ at 0x102ce1bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1028c5bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
